# Spatial Analysis (Data with exact locations)

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from haversine import haversine
from geopy.distance import geodesic

In [2]:
loc_avail_df = pd.read_csv("../Dataset/New_Data/9_location_data_corrected.csv", low_memory=False)

In [3]:
# Remove data points outside sri lanka
loc_avail_df = loc_avail_df[(loc_avail_df['lat'] >= 5.55) & (loc_avail_df['lat'] <= 9.51)]
loc_avail_df = loc_avail_df[(loc_avail_df['lng'] >= 79.41) & (loc_avail_df['lng'] <= 81.53)]

In [4]:
loc_avail_df

,ad_id,UID,street,city,heading,desc,posted_date,price,price_type,price_land_pp,...,city_name,alt_name,city_lat,city_lon,g_city_lat,g_city_lon,main_city,main_city_id,Gen_land_price_PP,geometry
0,230585,75373,Buthgamuwe Road,Rajagiriya,13 Perch Land - Rajagiriya - Buthgamuwe Road,13.4 Perch Land - Immediate Sale <br />\nPrice...,1/2/2018 11:13,1600000,Per Perch,1600000.0,...,Rajagiriya,NaN,6.912780,79.893898,6.909411,79.894257,rajagiriya,5.0,NaN,POINT (79.911289 6.914671)
1,231969,85338,Siriwardena Road,Dehiwala,Land in Dehiwela close to Hill Street,Split Level 6.69 perches of land facing Siriwa...,1/13/2018 12:05,3000000,Per Perch,3000000.0,...,Dehiwala,Dehiwela,6.830119,79.880081,6.830119,79.880081,dehiwala,4.0,NaN,POINT (79.8749844 6.8488636)
2,231889,84432,Watareka,Homagama,Land for sale in Homagama,Ã Â·âÃ Â·ÂÃ Â¶Â¸Ã Â·ÂÃ Â¶ÅÃ Â¶Â¸ Ã Â·â¬Ã...,1/12/2018 11:59,285000,Per Perch,285000.0,...,Homagama,NaN,6.844007,80.005394,6.844007,80.005394,homagama,9.0,NaN,POINT (80.08082 6.855431)
3,234384,87065,Robert Gunawardana Mw,Malabe,10 P Residential Land Plots for Sale on Robert...,10 P highly residential land plots for sale in...,2/1/2018 17:33,1000000,Per Perch,1000000.0,...,Malabe,Malambe,6.906079,79.969627,6.906079,79.969627,malabe,8.0,NaN,POINT (79.960866 6.906933)
4,233356,91020,Old Kottawa Road,Mirihana,Land for Sale in Mirihana,Bare land for sale in Mirihana facing Old Kott...,1/24/2018 15:21,3250000,Per Perch,3250000.0,...,Mirihana,NaN,6.868965,79.903877,6.868965,79.903877,nugegoda,1.0,NaN,POINT (79.904349 6.873143)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10348,5446309,195467,Mel road,Katubedda,à¶à¶§à·à¶¶à·à¶¯à·à¶¯à·à¶±à· à¶´à·à¶» à¶...,à¶ à·à·à·à¶®à·à· à¶¸à¶ à·à·à¶»à¶à¶±à·...,3/11/2023 4:22,1000000,Per Perch,1000000.0,...,Katubedda,NaN,6.801460,79.899696,6.801457,79.899681,moratuwa,6.0,1000000.0,POINT (79.8996789 6.8014572)
10349,5446532,139136,NaN,Homagama,27 perches bare land for sale in Homagama for ...,1.5km to Homagama town <br />\n4km to Athurugi...,3/11/2023 11:32,1325000,Per Perch,1325000.0,...,Homagama,NaN,6.844007,80.005394,6.844007,80.005394,homagama,9.0,1325000.0,POINT (80.00421327 6.855419783)
10350,5446655,125887,Rathnarama mawatha,Malabe,59 perches bare land for sale in Malabe for Rs...,59 perches land<br />\nTwo storrey building<br...,3/11/2023 1:16,1550000,Per Perch,1550000.0,...,Malabe,Malambe,6.906079,79.969627,6.906079,79.969627,malabe,8.0,1550000.0,POINT (79.97200855 6.892406735)
10351,5446833,120689,Malabe,Malabe,Commercial Land For Sale,"Address: Kaduwela Main Road (New Kandy Road), ...",3/12/2023 2:49,23250000,Per Perch,23250000.0,...,Malabe,Malambe,6.906079,79.969627,6.906079,79.969627,malabe,8.0,23250000.0,POINT (79.95980277 6.903916543)


In [5]:
loc_avail_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9980 entries, 0 to 10352
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ad_id              9980 non-null   int64  
 1   UID                9980 non-null   int64  
 2   street             5427 non-null   object 
 3   city               9980 non-null   object 
 4   heading            9980 non-null   object 
 5   desc               9980 non-null   object 
 6   posted_date        9980 non-null   object 
 7   price              9980 non-null   int64  
 8   price_type         8860 non-null   object 
 9   price_land_pp      9979 non-null   float64
 10  price_land_pa      9979 non-null   float64
 11  price_land_total   9979 non-null   float64
 12  agent_ref          4344 non-null   object 
 13  contact_type       9911 non-null   object 
 14  contact_name       9932 non-null   object 
 15  email              9980 non-null   object 
 16  lat                9980 non-

In [6]:
WGS1864_crs = 'EPSG:4326'
building_points_data = gpd.read_file('../Dataset/Shape_Files/DD_2023_214/DD_2023_214/COLOMBO_DISTRICT_CUT/CO_Building_Pt.shp').to_crs(WGS1864_crs)

In [7]:
building_points_data

,BU_Nm,BU_Lb_meth,BU_Ye_crea,BU_Lb_me_1,BU_Ye_revi,BU_Cd_ID,BU_GFCODE,geometry
0,NaN,9,2017,NaN,0,7309273,BLDGA,POINT (79.94422 6.77403)
1,Railway Station - Moratuwa,9,2015,NaN,0,7300078,RLSTA,POINT (79.88217 6.77392)
2,St. Sebastian's Church,8,2000,NaN,0,6601777,CHRHA,POINT (79.88326 6.77408)
3,NaN,9,2016,NaN,0,6604121,BLDGA,POINT (79.92498 6.77435)
4,Railway Station Moratuwa,8,2000,NaN,0,6601857,RLSTA,POINT (79.88179 6.77429)
...,...,...,...,...,...,...,...,...
18049,NaN,9,2017,NaN,0,7401236,BLDGA,POINT (80.11377 6.77194)
18050,NaN,9,2017,NaN,0,7401292,BLDGA,POINT (80.12385 6.77281)
18051,NaN,9,2017,NaN,0,7401230,BLDGA,POINT (80.11078 6.77316)
18052,NaN,9,2017,NaN,0,7401293,BLDGA,POINT (80.12348 6.77337)


In [8]:
print(building_points_data.crs)

EPSG:4326


In [9]:
# Create a GeoDataFrame from the CSV data
csv_geo_df = gpd.GeoDataFrame(loc_avail_df, geometry=gpd.points_from_xy(loc_avail_df.lng, loc_avail_df.lat))

In [10]:
csv_geo_df

,ad_id,UID,street,city,heading,desc,posted_date,price,price_type,price_land_pp,...,city_name,alt_name,city_lat,city_lon,g_city_lat,g_city_lon,main_city,main_city_id,Gen_land_price_PP,geometry
0,230585,75373,Buthgamuwe Road,Rajagiriya,13 Perch Land - Rajagiriya - Buthgamuwe Road,13.4 Perch Land - Immediate Sale <br />\nPrice...,1/2/2018 11:13,1600000,Per Perch,1600000.0,...,Rajagiriya,NaN,6.912780,79.893898,6.909411,79.894257,rajagiriya,5.0,NaN,POINT (79.91129 6.91467)
1,231969,85338,Siriwardena Road,Dehiwala,Land in Dehiwela close to Hill Street,Split Level 6.69 perches of land facing Siriwa...,1/13/2018 12:05,3000000,Per Perch,3000000.0,...,Dehiwala,Dehiwela,6.830119,79.880081,6.830119,79.880081,dehiwala,4.0,NaN,POINT (79.87498 6.84886)
2,231889,84432,Watareka,Homagama,Land for sale in Homagama,Ã Â·âÃ Â·ÂÃ Â¶Â¸Ã Â·ÂÃ Â¶ÅÃ Â¶Â¸ Ã Â·â¬Ã...,1/12/2018 11:59,285000,Per Perch,285000.0,...,Homagama,NaN,6.844007,80.005394,6.844007,80.005394,homagama,9.0,NaN,POINT (80.08082 6.85543)
3,234384,87065,Robert Gunawardana Mw,Malabe,10 P Residential Land Plots for Sale on Robert...,10 P highly residential land plots for sale in...,2/1/2018 17:33,1000000,Per Perch,1000000.0,...,Malabe,Malambe,6.906079,79.969627,6.906079,79.969627,malabe,8.0,NaN,POINT (79.96087 6.90693)
4,233356,91020,Old Kottawa Road,Mirihana,Land for Sale in Mirihana,Bare land for sale in Mirihana facing Old Kott...,1/24/2018 15:21,3250000,Per Perch,3250000.0,...,Mirihana,NaN,6.868965,79.903877,6.868965,79.903877,nugegoda,1.0,NaN,POINT (79.90435 6.87314)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10348,5446309,195467,Mel road,Katubedda,à¶à¶§à·à¶¶à·à¶¯à·à¶¯à·à¶±à· à¶´à·à¶» à¶...,à¶ à·à·à·à¶®à·à· à¶¸à¶ à·à·à¶»à¶à¶±à·...,3/11/2023 4:22,1000000,Per Perch,1000000.0,...,Katubedda,NaN,6.801460,79.899696,6.801457,79.899681,moratuwa,6.0,1000000.0,POINT (79.89968 6.80146)
10349,5446532,139136,NaN,Homagama,27 perches bare land for sale in Homagama for ...,1.5km to Homagama town <br />\n4km to Athurugi...,3/11/2023 11:32,1325000,Per Perch,1325000.0,...,Homagama,NaN,6.844007,80.005394,6.844007,80.005394,homagama,9.0,1325000.0,POINT (80.00421 6.85542)
10350,5446655,125887,Rathnarama mawatha,Malabe,59 perches bare land for sale in Malabe for Rs...,59 perches land<br />\nTwo storrey building<br...,3/11/2023 1:16,1550000,Per Perch,1550000.0,...,Malabe,Malambe,6.906079,79.969627,6.906079,79.969627,malabe,8.0,1550000.0,POINT (79.97201 6.89241)
10351,5446833,120689,Malabe,Malabe,Commercial Land For Sale,"Address: Kaduwela Main Road (New Kandy Road), ...",3/12/2023 2:49,23250000,Per Perch,23250000.0,...,Malabe,Malambe,6.906079,79.969627,6.906079,79.969627,malabe,8.0,23250000.0,POINT (79.95980 6.90392)


In [11]:
# # Create a new column for each target point type to store the smallest distance
# for target_point_type in target_point_types:
#     target_points = building_points_data[building_points_data['BU_GFCODE'] == target_point_type]
#     csv_geo_df[f'smallest_distance_to_{target_point_type}'] = csv_geo_df['geometry'].apply(
#         lambda x: min(
#             haversine((x.y, x.x), (point.geometry.y, point.geometry.x))
#             for _, point in target_points.iterrows()
#         )
#     )

### Select types of places from the information given by Survey Department

In [12]:
# hospitals
target_hospital_types = ['BHSPP', 'DHSPA', 'GHSPA', 'PHSPA', 'RHSPA', 'THSPA', 'UHSPA']  

In [13]:
csv_geo_df['dist_Hospital'] = csv_geo_df['geometry'].apply(
    lambda x: min(
        geodesic((x.y, x.x), (point.y, point.x)).kilometers
        for point in building_points_data.loc[building_points_data['BU_GFCODE'].isin(target_hospital_types), 'geometry']
    )
)

In [14]:
# Railway stations
target_railway_station_types = ['RLHTA', 'RLSTA']  

In [15]:
csv_geo_df['dist_rail_station'] = csv_geo_df['geometry'].apply(
    lambda x: min(
        geodesic((x.y, x.x), (point.y, point.x)).kilometers
        for point in building_points_data.loc[building_points_data['BU_GFCODE'].isin(target_railway_station_types), 'geometry']
    )
)

In [16]:
# bus stop
target_bus_stop_types = ['BUSHA', 'BUSSA']  

In [17]:
csv_geo_df['dist_bus_stop'] = csv_geo_df['geometry'].apply(
    lambda x: min(
        geodesic((x.y, x.x), (point.y, point.x)).kilometers
        for point in building_points_data.loc[building_points_data['BU_GFCODE'].isin(target_bus_stop_types), 'geometry']
    )
)

In [18]:
# Post office
target_post_office_types = ['APOFA', 'MPOFA', 'SPOFA']  

In [19]:
csv_geo_df['dist_post_office'] = csv_geo_df['geometry'].apply(
    lambda x: min(
        geodesic((x.y, x.x), (point.y, point.x)).kilometers
        for point in building_points_data.loc[building_points_data['BU_GFCODE'].isin(target_post_office_types), 'geometry']
    )
)

In [20]:
other_types = ['GVTBA', 'PLCSA', 'NSCHA']

In [21]:
# Create a new column for each target point type to store the smallest distance
for other_types in other_types:
    target_points = building_points_data[building_points_data['BU_GFCODE'] == other_types]
    csv_geo_df[f'smallest_distance_to_{other_types}'] = csv_geo_df['geometry'].apply(
        lambda x: min(
            haversine((x.y, x.x), (point.geometry.y, point.geometry.x))
            for _, point in target_points.iterrows()
        )
    )

In [22]:
column_mapping = {
    'smallest_distance_to_GVTBA': 'dist_government_office',
    'smallest_distance_to_PLCSA': 'dist_police_station',
    'smallest_distance_to_NSCHA': 'dist_national_school'
}

csv_geo_df = csv_geo_df.rename(columns=column_mapping)

In [23]:
# Schools
target_point_types = ['CSCHA', 'MSCHA', 'OSCHA', 'PSCHA', 'USCHA']  

In [24]:
csv_geo_df['dist_other_School'] = csv_geo_df['geometry'].apply(
    lambda x: min(
        geodesic((x.y, x.x), (point.y, point.x)).kilometers
        for point in building_points_data.loc[building_points_data['BU_GFCODE'].isin(target_point_types), 'geometry']
    )
)

In [25]:
# Save the resulting GeoDataFrame to a new CSV file
output_csv_path = '../Dataset/New_Data/10_dist_cal.csv' 
csv_geo_df.to_csv(output_csv_path, index=False)

In [26]:
point_dist_cal_df = pd.read_csv("../Dataset/New_Data/10_dist_cal.csv", low_memory=False)

In [31]:
# Read your shapefile with lines (replace 'your_lines.shp' with the actual file path)
for_main_road_dist = gpd.read_file('../Dataset/Shape_Files/Created_New/Within_Colombo_only.shp')

for_main_road_dist

,ad_id,UID,street,city,heading,desc_,posted_dat,price,price_type,price_land,...,dist_bus_s,dist_post_,dist_gover,dist_polic,dist_natio,dist_other,x_sld99,y_sld99,NEAR_DIST,geometry
0,230585,75373,Buthgamuwe Road,Rajagiriya,13 Perch Land - Rajagiriya - Buthgamuwe Road,13.4 Perch Land - Immediate Sale <br />\nPrice...,2018-01-02,1600000,Per Perch,1600000.0,...,2.964581,1.686405,0.904207,1.870336,1.900797,0.690246,404691.684670,490562.385683,5.946677,POINT (404691.685 490562.386)
1,231969,85338,Siriwardena Road,Dehiwala,Land in Dehiwela close to Hill Street,Split Level 6.69 perches of land facing Siriwa...,2018-01-13,3000000,Per Perch,3000000.0,...,1.934521,1.218802,0.793445,0.374929,8.006813,0.311494,400665.656240,483291.970431,119.256538,POINT (400665.656 483291.970)
2,231889,84432,Watareka,Homagama,Land for sale in Homagama,à·„à·à¶¸à·à¶œà¶¸ à·€à¶§à¶»à·à¶š à·„à¶ºà·’à¶...,2018-01-12,285000,Per Perch,285000.0,...,8.847441,2.060439,1.397882,1.949055,5.777922,0.549640,423417.875944,483980.423403,135.197494,POINT (423417.876 483980.423)
3,234384,87065,Robert Gunawardana Mw,Malabe,10 P Residential Land Plots for Sale on Robert...,10 P highly residential land plots for sale in...,2018-02-01,1000000,Per Perch,1000000.0,...,4.537480,0.822716,0.790496,3.508479,4.167544,0.482050,410169.300850,489696.954980,250.725410,POINT (410169.301 489696.955)
4,233356,91020,Old Kottawa Road,Mirihana,Land for Sale in Mirihana,Bare land for sale in Mirihana facing Old Kott...,2018-01-24,3250000,Per Perch,3250000.0,...,1.454383,1.612721,1.188002,0.287015,3.881640,0.690426,403916.301455,485971.096659,5.275334,POINT (403916.301 485971.097)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,5446309,195467,Mel road,Katubedda,කටුබැද්දෙන් පාර අයිනේ වටිනා ඉඩමක්,අවස්ථාව මග හැරගන්න එපා...<br />\nමේ ඔබට ලැබෙන ...,2023-03-11,1000000,Per Perch,1000000.0,...,2.648356,2.708873,1.260597,1.655175,7.669409,0.508840,403385.682269,478044.129815,658.682153,POINT (403385.682 478044.130)
9976,5446532,139136,NaN,Homagama,27 perches bare land for sale in Homagama for ...,1.5km to Homagama town <br />\n4km to Athurugi...,2023-03-11,1325000,Per Perch,1325000.0,...,1.570572,2.931833,1.385384,2.075257,5.197747,0.894467,414950.717083,483992.132231,8.267908,POINT (414950.717 483992.132)
9977,5446655,125887,Rathnarama mawatha,Malabe,59 perches bare land for sale in Malabe for Rs...,59 perches land<br />\nTwo storrey building<br...,2023-03-11,1550000,Per Perch,1550000.0,...,2.534638,2.320759,2.300348,3.476300,5.484368,0.110846,411398.038512,488088.390318,752.870005,POINT (411398.039 488088.390)
9978,5446833,120689,Malabe,Malabe,Commercial Land For Sale,"Address: Kaduwela Main Road (New Kandy Road), ...",2023-03-12,23250000,Per Perch,23250000.0,...,4.388711,0.617471,0.596578,3.453980,4.023999,0.253730,410051.223287,489363.562971,4.868584,POINT (410051.223 489363.563)


In [32]:
for_water_stream_dist = gpd.read_file('../Dataset/Shape_Files/Created_New/Within_Colombo_only - Copy.shp')

for_water_stream_dist

,ad_id,UID,street,city,heading,desc_,posted_dat,price,price_type,price_land,...,dist_bus_s,dist_post_,dist_gover,dist_polic,dist_natio,dist_other,x_sld99,y_sld99,NEAR_DIST,geometry
0,230585,75373,Buthgamuwe Road,Rajagiriya,13 Perch Land - Rajagiriya - Buthgamuwe Road,13.4 Perch Land - Immediate Sale <br />\nPrice...,2018-01-02,1600000,Per Perch,1600000.0,...,2.964581,1.686405,0.904207,1.870336,1.900797,0.690246,404691.684670,490562.385683,218.669877,POINT (404691.685 490562.386)
1,231969,85338,Siriwardena Road,Dehiwala,Land in Dehiwela close to Hill Street,Split Level 6.69 perches of land facing Siriwa...,2018-01-13,3000000,Per Perch,3000000.0,...,1.934521,1.218802,0.793445,0.374929,8.006813,0.311494,400665.656240,483291.970431,724.454534,POINT (400665.656 483291.970)
2,231889,84432,Watareka,Homagama,Land for sale in Homagama,à·„à·à¶¸à·à¶œà¶¸ à·€à¶§à¶»à·à¶š à·„à¶ºà·’à¶...,2018-01-12,285000,Per Perch,285000.0,...,8.847441,2.060439,1.397882,1.949055,5.777922,0.549640,423417.875944,483980.423403,89.933025,POINT (423417.876 483980.423)
3,234384,87065,Robert Gunawardana Mw,Malabe,10 P Residential Land Plots for Sale on Robert...,10 P highly residential land plots for sale in...,2018-02-01,1000000,Per Perch,1000000.0,...,4.537480,0.822716,0.790496,3.508479,4.167544,0.482050,410169.300850,489696.954980,1262.706775,POINT (410169.301 489696.955)
4,233356,91020,Old Kottawa Road,Mirihana,Land for Sale in Mirihana,Bare land for sale in Mirihana facing Old Kott...,2018-01-24,3250000,Per Perch,3250000.0,...,1.454383,1.612721,1.188002,0.287015,3.881640,0.690426,403916.301455,485971.096659,410.043706,POINT (403916.301 485971.097)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,5446309,195467,Mel road,Katubedda,කටුබැද්දෙන් පාර අයිනේ වටිනා ඉඩමක්,අවස්ථාව මග හැරගන්න එපා...<br />\nමේ ඔබට ලැබෙන ...,2023-03-11,1000000,Per Perch,1000000.0,...,2.648356,2.708873,1.260597,1.655175,7.669409,0.508840,403385.682269,478044.129815,248.041042,POINT (403385.682 478044.130)
9976,5446532,139136,NaN,Homagama,27 perches bare land for sale in Homagama for ...,1.5km to Homagama town <br />\n4km to Athurugi...,2023-03-11,1325000,Per Perch,1325000.0,...,1.570572,2.931833,1.385384,2.075257,5.197747,0.894467,414950.717083,483992.132231,1435.686814,POINT (414950.717 483992.132)
9977,5446655,125887,Rathnarama mawatha,Malabe,59 perches bare land for sale in Malabe for Rs...,59 perches land<br />\nTwo storrey building<br...,2023-03-11,1550000,Per Perch,1550000.0,...,2.534638,2.320759,2.300348,3.476300,5.484368,0.110846,411398.038512,488088.390318,673.707551,POINT (411398.039 488088.390)
9978,5446833,120689,Malabe,Malabe,Commercial Land For Sale,"Address: Kaduwela Main Road (New Kandy Road), ...",2023-03-12,23250000,Per Perch,23250000.0,...,4.388711,0.617471,0.596578,3.453980,4.023999,0.253730,410051.223287,489363.562971,1265.663064,POINT (410051.223 489363.563)


In [33]:
dist_cal_compelete = for_main_road_dist
new_column_data = for_water_stream_dist['NEAR_DIST'].copy()
dist_cal_compelete['dist_water'] = new_column_data

dist_cal_compelete = dist_cal_compelete.rename(columns={'NEAR_DIST': 'dist_road'})

In [37]:
dist_cal_compelete = dist_cal_compelete.drop(columns=['geometry'])

dist_cal_compelete

,ad_id,UID,street,city,heading,desc_,posted_dat,price,price_type,price_land,...,dist_bus_s,dist_post_,dist_gover,dist_polic,dist_natio,dist_other,x_sld99,y_sld99,dist_road,dist_water
0,230585,75373,Buthgamuwe Road,Rajagiriya,13 Perch Land - Rajagiriya - Buthgamuwe Road,13.4 Perch Land - Immediate Sale <br />\nPrice...,2018-01-02,1600000,Per Perch,1600000.0,...,2.964581,1.686405,0.904207,1.870336,1.900797,0.690246,404691.684670,490562.385683,5.946677,218.669877
1,231969,85338,Siriwardena Road,Dehiwala,Land in Dehiwela close to Hill Street,Split Level 6.69 perches of land facing Siriwa...,2018-01-13,3000000,Per Perch,3000000.0,...,1.934521,1.218802,0.793445,0.374929,8.006813,0.311494,400665.656240,483291.970431,119.256538,724.454534
2,231889,84432,Watareka,Homagama,Land for sale in Homagama,à·„à·à¶¸à·à¶œà¶¸ à·€à¶§à¶»à·à¶š à·„à¶ºà·’à¶...,2018-01-12,285000,Per Perch,285000.0,...,8.847441,2.060439,1.397882,1.949055,5.777922,0.549640,423417.875944,483980.423403,135.197494,89.933025
3,234384,87065,Robert Gunawardana Mw,Malabe,10 P Residential Land Plots for Sale on Robert...,10 P highly residential land plots for sale in...,2018-02-01,1000000,Per Perch,1000000.0,...,4.537480,0.822716,0.790496,3.508479,4.167544,0.482050,410169.300850,489696.954980,250.725410,1262.706775
4,233356,91020,Old Kottawa Road,Mirihana,Land for Sale in Mirihana,Bare land for sale in Mirihana facing Old Kott...,2018-01-24,3250000,Per Perch,3250000.0,...,1.454383,1.612721,1.188002,0.287015,3.881640,0.690426,403916.301455,485971.096659,5.275334,410.043706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,5446309,195467,Mel road,Katubedda,කටුබැද්දෙන් පාර අයිනේ වටිනා ඉඩමක්,අවස්ථාව මග හැරගන්න එපා...<br />\nමේ ඔබට ලැබෙන ...,2023-03-11,1000000,Per Perch,1000000.0,...,2.648356,2.708873,1.260597,1.655175,7.669409,0.508840,403385.682269,478044.129815,658.682153,248.041042
9976,5446532,139136,NaN,Homagama,27 perches bare land for sale in Homagama for ...,1.5km to Homagama town <br />\n4km to Athurugi...,2023-03-11,1325000,Per Perch,1325000.0,...,1.570572,2.931833,1.385384,2.075257,5.197747,0.894467,414950.717083,483992.132231,8.267908,1435.686814
9977,5446655,125887,Rathnarama mawatha,Malabe,59 perches bare land for sale in Malabe for Rs...,59 perches land<br />\nTwo storrey building<br...,2023-03-11,1550000,Per Perch,1550000.0,...,2.534638,2.320759,2.300348,3.476300,5.484368,0.110846,411398.038512,488088.390318,752.870005,673.707551
9978,5446833,120689,Malabe,Malabe,Commercial Land For Sale,"Address: Kaduwela Main Road (New Kandy Road), ...",2023-03-12,23250000,Per Perch,23250000.0,...,4.388711,0.617471,0.596578,3.453980,4.023999,0.253730,410051.223287,489363.562971,4.868584,1265.663064


In [38]:
output_csv_path = '../Dataset/New_Data/11_dist_cal_completed.csv' 
dist_cal_compelete.to_csv(output_csv_path, index=False)